# Import of a PCB and its components via IDF and EDB

This example shows how to import a PCB and its components using IDF files (LDB and BDF).
You can also use a combination of EMN and EMP files in a similar way.

Keywords: **Icepak**, **PCB**, **IDF**.

In [1]:
# ## Perform imports and define constants
#
# Perform required imports.

In [2]:
import os
import tempfile
import time

from ansys.aedt.core.examples.downloads import download_file
from ansys.aedt.core import Hfss3dLayout, Icepak

Define constants.

In [3]:
AEDT_VERSION = "2025.2"
NG_MODE = False  # Open AEDT UI when it is launched.

## Open project

Open an empty project in non-graphical mode, using a temporary folder.

In [4]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

ipk = Icepak(
    project=os.path.join(temp_folder.name, "Icepak_ECAD_Import.aedt"),
    version=AEDT_VERSION,
    new_desktop=True,
    non_graphical=NG_MODE,
)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_7316ad35-c1e7-4cc1-8f1a-ab7ed6ccb081.log is enabled.


...

PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project Icepak_ECAD_Import has been created.


PyAEDT INFO: No design is present. Inserting a new design.


PyAEDT INFO: Added design 'Icepak_ACF' of type Icepak.


PyAEDT INFO: Aedt Objects correctly read


## Import the IDF files

Import the BDF and LDF files. Sample files are shown here.

<img src="_static\bdf.png" width="400">

<img src="_static\ldf.png" width="400">

You can import the IDF files with several filtering options, including caps, resistors,
inductors, power, specific power, and size.
There are also options for PCB creation, including number of layers, copper percentages, and layer sizes.

This example uses the default values for the PCB.
The imported PCB (from the IDF files) are deleted later and replaced by a PCB that has the trace
information (from ECAD) for higher accuracy.

Download ECAD and IDF files.

In [5]:
def_path = download_file(
    source="icepak/Icepak_ECAD_Import/A1_uprev.aedb",
    name="edb.def",
    local_path=temp_folder.name,
)
board_path = download_file(
    source="icepak/Icepak_ECAD_Import/", name="A1.bdf", local_path=temp_folder.name
)
library_path = download_file(
    source="icepak/Icepak_ECAD_Import/", name="A1.ldf", local_path=temp_folder.name
)

Import IDF file.

In [6]:
ipk.import_idf(board_path=board_path)

PyAEDT INFO: Active Design set to Icepak_ACF


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


PyAEDT INFO: Active Design set to Icepak_ACF


True

Save the project

In [7]:
ipk.save_project()

PyAEDT INFO: Project Icepak_ECAD_Import Saved correctly


True

## Import ECAD
Add an HFSS 3D Layout design with the layout information of the PCB.

In [8]:
hfss3d_lo = Hfss3dLayout(project=def_path, version=AEDT_VERSION)
hfss3d_lo.save_project()

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 7632!


PyAEDT INFO: EDB folder C:\Users\ansys\AppData\Local\Temp\tmp4czfis9p.ansys\icepak\Icepak_ECAD_Import\A1_uprev.aedb\edb.def has been imported to project A1_uprev


PyAEDT INFO: Active Design set to 0;A1_uprev


PyAEDT INFO: Active Design set to 0;A1_uprev


PyAEDT INFO: Aedt Objects correctly read


PyAEDT INFO: Project A1_uprev Saved correctly


True

Create a PCB component in Icepak linked to the HFSS 3D Layout project. The polygon ``"poly_0"``
is used as the outline of the PCB, and a dissipation of ``"1W"`` is applied to the PCB.

In [9]:
project_file = hfss3d_lo.project_file
design_name = hfss3d_lo.design_name

In [10]:
ipk.create_pcb_from_3dlayout(
    component_name="PCB_pyAEDT",
    project_name=project_file,
    design_name=design_name,
    extent_type="Polygon",
    outline_polygon="poly_0",
    power_in=1,
    close_linked_project_after_import=False,
)

PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 1sec


PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: PCB component correctly created in Icepak.


PCB_pyAEDT1

Delete the simplified PCB object coming from the IDF import.

In [11]:
ipk.modeler.delete_objects_containing("IDF_BoardOutline", False)

PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 1sec


PyAEDT INFO: Deleted 1 Objects: IDF_BoardOutline.


PyAEDT INFO: Deleted 1 objects


True

## Release AEDT

In [12]:
ipk.save_project()
ipk.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project Icepak_ECAD_Import Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [13]:
temp_folder.cleanup()